In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from scipy.stats import rv_continuous, norm, truncnorm, arcsine, chi2, gamma, triang, entropy
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from time import time
import operator as op

def nck(n, k):
    "Binomial coefficient n choose k"
    k = min(k, n-k)
    if k == 0: return 1
    numer = denom = 1
    for i in range(n-k+1, n+1):
        numer *= i
    for i in range(1, k+1):
        denom *=i
    return numer//denom

def ncmwr(n,m):
    "Number of ways to choose m objects from the set n with replacement"
    return nck(n+m-1, m)

class glove_distribution_gen(rv_continuous):
    def _pdf(self,x):
        conds = [x<0, (x>=0) & (x<1), (x>=1) & (x<3), (x>=3) & (x<4), x>4]
        funcs = [0, 0.7, 0, 0.3, 0]
        return np.piecewise(x, conds, funcs)
    def _cdf(self,x):
        conds = [x<0, (x>=0) & (x<1), (x>=1) & (x<3), (x>=3) & (x<4), x>4]
        funcs = [0, lambda x: 0.7*x, 0.7, lambda x: 0.7+0.3*(x-3), 1]
        return np.piecewise(x, conds, funcs)
    def _stats(self):
        return 1.4, 1.973333333333333, None, None
glove_distribution = glove_distribution_gen(name='glove_distribution')


gift_df = pd.read_csv('gifts.csv')
gift_list = gift_df.GiftId.unique()
gift_types = ['horse', 'ball', 'bike', 'train', 'coal', 'book', 'doll', 'blocks', 'gloves']
gift_dist_list = [
    truncnorm(a=(0-5)/2, b=(50-5)/2, loc=5, scale=2),
    truncnorm((0-2)/0.3, (50-2)/0.3, loc=2, scale=0.3),
    truncnorm((0-20)/10, (50-10)/10, loc=20, scale=10),
    truncnorm((0-10)/5, (50-10)/5, loc=10, scale=5),
    arcsine(scale=47),
    chi2(2),
    gamma(5),
    triang(c = 1/3, loc=5, scale=15),
    glove_distribution]

gift_numbers = [1000, 1100, 500, 1000, 166, 1200, 1000, 1000, 200]
gift_ids = {
    'horse' : [],
    'ball'  : [],
    'bike'  : [],
    'train' : [],
    'coal'  : [],
    'book'  : [],
    'doll'  : [],
    'blocks' : [],
    'gloves': []
}
count = 0
for i, gt in enumerate(gift_types):
    n_gifts = gift_numbers[i]
    for j in range(count, n_gifts+count):
        gift_ids[gt].append(gift_list[j])
    count += n_gifts
    
    
    gift_dists = {
    'horse' : truncnorm(a=(0-5)/2, b=(50-5)/2, loc=5, scale=2), #a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
    'ball'  : truncnorm((0-2)/0.3, (50-2)/0.3, loc=2, scale=0.3),
    'bike'  : truncnorm((0-20)/10, (50-10)/10, loc=20, scale=10),
    'train' : truncnorm((0-10)/5, (50-10)/5, loc=10, scale=5),
    'coal'  : arcsine(scale=47),
    'book'  : chi2(2),
    'doll'  : gamma(5),
    'blocks' : triang(c = 1/3, loc=5, scale=15),
    'gloves': glove_distribution
}

toy_index = {
    0 : 'horse',
    1 : 'ball',
    2 : 'bike',
    3 : 'train',
    4 : 'coal',
    5 : 'book',
    6 : 'doll',
    7 : 'blocks',
    8 : 'gloves'  
}

In [91]:
def isiterable(obj):
    try:
        iter(obj)
    except TypeError:
        return False 
    else:
        return True

class combination:
    def __init__(self, iterable, items = None):
        #combination = {items : quantity}
        #items = [first_item, second_item, ...]
        self.length = 0
        if items == None:
            raise Exception('items must be provided.')
        else:
            self.comb = dict(zip(sorted(items), [0] * len(items)))
            for i in iterable:
                self.comb[i] += 1
                self.length += 1
        self.key = frozenset(self.comb.items())
    
    def __eq__(self, other):
        """Override the default Equals behavior"""
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        return NotImplemented
    
    def __ne__(self, other):
        """Define a non-equality test"""
        if isinstance(other, self.__class__):
            return not self.__eq__(other)
        return NotImplemented
    
    def __hash__(self):
        """Override the default hash behavior (that returns the id or the object)"""
        return hash(self.key)
    
    def __repr__(self):
        return str(self.get_list())
    
    def get_len(self):
        return self.length
    
    def get_list(self):
        out = []
        for key, value in self.comb.items():
            for i in range(value):
                out.append(key)
        return sorted(out)
    
    def contains(self, other):
        #checks if self contains other combination
        self_d = self.comb.copy()
        
        for item, qty in self_d.items():
            if item not in other.comb: other.comb[item] = 0
            elif self_d[item] < other.comb[item]: return False
        return True
    
    def distance(self, other):
        d = 0
        for key in self.comb:
            d += np.absolute(self.comb[key] - other.comb[key])
        return d
    
    def difference(self, other):
        #self must contain other
        if not self.contains(other):
            raise Exception('Self must contain other.')
        dif = []
        items = [i for i in self.comb]
        for key in self.comb:
            for i in range(self.comb[key] - other.comb[key]):
                dif.append(combination([key], items))
        return dif
    
    def get_comb(self):
        return self.comb
    
    def get_str(self):
        return self.__repr__()
    
    def get_unique_items(self):
        for item in self.comb:
            if self.comb[item] > 0:
                yield item

def test_isvalid(comb, rules_list):
    #returns True if combination is valid according to the rules
    #print('combination = %s' % (comb.get_str()))
    for rule in rules_list:
        if comb.contains(rule):
            return False
    return True


class convolutions_galore:
    def __init__(self, distribution_names, distributions, N=201, limits=[0, 50]):
        self.keys = distribution_names
        self.x = np.linspace(limits[0], limits[1], N)
        self.delta = (limits[1]-limits[0])/(N-1)
        self.packs_pdfs = {}
        self.rules = []
        for index, item in enumerate(self.keys):
            dist_pdf = self._no_inf_pdf(distributions[index])
            dist_pdf = self._kill_trailing_zeroes(dist_pdf)
            self.packs_pdfs[combination([item], self.keys)] = [self._expected_score(dist_pdf), np.asarray(dist_pdf)]

    def _cut_pdf(self, pdf):
        n = len(self.x)
        if len(pdf) <= n:
            return pdf
        else:
            return pdf[0:n]
    
   
    @staticmethod
    def _kill_trailing_zeroes(mahlist):
        for i in reversed(range(len(mahlist))):
            if mahlist[i] <= 1e-50:
                pass
            else:
                mahlist = mahlist[:i+1]
                break
        return mahlist
    
    
    def _no_inf_pdf(self, dist):
        #get the distribution in the range
        dist_pdf = dist.pdf(self.x)
        #get all the infinities
        inf_index = np.where(dist_pdf == np.inf)[0]
        #check if there are any infinities
        if len(inf_index) == 0:
            return dist_pdf #no infinities, phew
        #treat infinities
        for i in inf_index:
            xi = self.x[i]
            dist_pdf[i] = (dist.cdf(xi + self.delta/2) - dist.cdf(xi - self.delta/2))/self.delta #hoping that there is a closed expression for the cdf
        return dist_pdf #no more infinities, phew    
    
    
    def _expected_score(self,pdf):
        es = 0
        pdf = self._cut_pdf(pdf)
        for index, y in enumerate(pdf):
            es += self.x[index] * y
        return es * self.delta
    
    
    def _find_lowest_distance_key(self, new_combination):
        #find the key with the lowest distance to the new combination
        lowest_distance = np.inf
        closest_key = None 
        for key in self.packs_pdfs:
            d = key.distance(new_combination)
            if d <= 1:
                return key, d
            elif  d < lowest_distance:
                lowest_distance = d
                closest_key = key
        return closest_key, d
    

    def _add_new_rule(self, new_combination):
        self.rules.append(new_combination)
    
    
    def _isvalid(self, new_combination):
        #returns True if combination is valid according to the rules
        for rule in self.rules:
            if new_combination.contains(rule):
                return False
        return True
    
    
    def add_new_convolution(self, new_combination):
        "Returns True if new_combination is successfully added or is already in, False otherwisely"
        [closest_key, closest_distance] = self._find_lowest_distance_key(new_combination)
        
        if not self._isvalid(new_combination):
            return False
        
        if closest_distance == 0:
            return True
        
        differences = new_combination.difference(closest_key)
        [old_score, old_package_pdf] = self.packs_pdfs[closest_key]
        new_package_pdf = old_package_pdf
        
        for key in differences:
            item_pdf = self.packs_pdfs[key][1]
            new_package_pdf = np.convolve(new_package_pdf, item_pdf)*self.delta
            new_package_pdf = self._kill_trailing_zeroes(new_package_pdf)
            
        new_score = self._expected_score(new_package_pdf)
        
        if new_score > old_score:
            self.packs_pdfs[new_combination] = [new_score, new_package_pdf]
            return True
        else:
            self._add_new_rule(new_combination)
            return False
    
    def find_all_valid_convolutions(self, depth=5, time_limit=600):
        then = time()
        for i in range(1, depth+1):
            print('Time elapsed: %.2f\n---\nCombination size: %d' % (time()-then, i))
            k = 0
            for cb in combinations_with_replacement(self.keys, i):
                c = combination(cb, self.keys)
                self.add_new_convolution(c)
                k+=1
                if k%10000 == 0:
                    if time()-then > time_limit:
                        return
    
    def remove_invalid_combinations(self, combinations_batch):
        for c in combinations_batch:
            if not self._isvalid(c):
                combinations_batch.remove(c)
    
    def add_convolutions_batch(self, combinations_batch):
        "Call self.add_new_convolution for every item in combinations_batch, returns list of unused items"
        unused_keys = self.keys[:]
        for c in combinations_batch:
            if self.add_new_convolution(c) and unused_keys:
                used_items = c.get_unique_items()
                for ui in used_items:
                    try:
                        unused_keys.remove(ui)
                    except:
                        pass
        return unused_keys
    
    def find_all_valid_convolutions_batch(self, depth=5, time_limit=600):
        then = time()
        internal_keys = self.keys[:]
        for i in range(1, depth+1):
            print('Time elapsed: %.2f\n---\nCombination size: %d' % (time()-then, i))
            combinations_batch = [combination(cb, self.keys) for cb in combinations_with_replacement(internal_keys, i)]
            self.remove_invalid_combinations(combinations_batch)
            unused_keys = self.add_convolutions_batch(combinations_batch)
            for ui in unused_keys:
                try:
                    internal_keys.remove(ui)
                except:
                    pass
            if unused_keys:
                print('=====')
                print(unused_keys)
                print(internal_keys)
                #print(combinations_batch)
                print('=====')

    
    

In [ ]:
then = time()
a = convolutions_galore(gift_types, gift_dist_list, N=5001)
a.find_all_valid_convolutions_batch(25)
for key, value in a.packs_pdfs.items():
    #print('Combination = %s, Score = %.2f' % (key, value[0]))
    pass
print('Time elapsed: %.2f' % (time()-then))
b = [[key, value[0], sum(value[1])*a.delta] for key, value in a.packs_pdfs.items()]
b.sort(key = lambda x: x[1])
print('Number of combinations: %d' % (len(a.packs_pdfs.items())))
print('Number of rules: %d' % (len(a.rules)))
b[-5:-1]

Time elapsed: 0.00
---
Combination size: 1
Time elapsed: 0.00
---
Combination size: 2
Time elapsed: 0.36
---
Combination size: 3
Time elapsed: 1.84
---
Combination size: 4
Time elapsed: 6.55
---
Combination size: 5
Time elapsed: 20.64
---
Combination size: 6
Time elapsed: 55.00
---
Combination size: 7
Time elapsed: 142.48
---
Combination size: 8
=====
['coal']
['horse', 'ball', 'bike', 'train', 'book', 'doll', 'blocks', 'gloves']
=====
Time elapsed: 395.33
---
Combination size: 9
=====
['coal']
['horse', 'ball', 'bike', 'train', 'book', 'doll', 'blocks', 'gloves']
=====
Time elapsed: 701.49
---
Combination size: 10
=====
['coal']
['horse', 'ball', 'bike', 'train', 'book', 'doll', 'blocks', 'gloves']
=====
Time elapsed: 1415.42
---
Combination size: 11
=====
['coal']
['horse', 'ball', 'bike', 'train', 'book', 'doll', 'blocks', 'gloves']
=====
Time elapsed: 2960.58
---
Combination size: 12
=====
['coal']
['horse', 'ball', 'bike', 'train', 'book', 'doll', 'blocks', 'gloves']
=====
Time el

In [90]:
c = combination(['bike'], gift_types)
[i for i in a.packs_pdfs if i.contains(c) and i.get_len()==7]

[['ball', 'ball', 'ball', 'ball', 'bike', 'doll', 'gloves'],
 ['ball', 'bike', 'gloves', 'gloves', 'gloves', 'gloves', 'horse'],
 ['bike', 'book', 'gloves', 'gloves', 'gloves', 'gloves', 'gloves'],
 ['ball', 'ball', 'bike', 'gloves', 'gloves', 'horse', 'horse'],
 ['ball', 'bike', 'book', 'gloves', 'gloves', 'gloves', 'gloves'],
 ['bike', 'book', 'book', 'book', 'doll', 'gloves', 'horse'],
 ['ball', 'ball', 'bike', 'blocks', 'book', 'book', 'book'],
 ['ball', 'ball', 'ball', 'ball', 'bike', 'doll', 'doll'],
 ['ball', 'ball', 'ball', 'bike', 'gloves', 'gloves', 'gloves'],
 ['bike', 'book', 'book', 'book', 'book', 'doll', 'gloves'],
 ['ball', 'bike', 'doll', 'doll', 'gloves', 'gloves', 'gloves'],
 ['bike', 'book', 'book', 'book', 'book', 'book', 'horse'],
 ['ball', 'bike', 'book', 'book', 'doll', 'gloves', 'gloves'],
 ['ball', 'ball', 'bike', 'doll', 'gloves', 'gloves', 'horse'],
 ['bike', 'book', 'book', 'book', 'book', 'book', 'gloves'],
 ['ball', 'bike', 'book', 'book', 'book', 'book',

In [7]:
then = time()
a = convolutions_galore(gift_types, gift_dist_list, N=1001)
a.find_all_valid_convolutions(6)
for key, value in a.packs_pdfs.items():
    #print('Combination = %s, Score = %.2f' % (key, value[0]))
    pass
print('Time elapsed: %.2f' % (time()-then))

Time elapsed: 0.00
---
Combination size: 1
Time elapsed: 0.00
---
Combination size: 2
Time elapsed: 0.06
---
Combination size: 3
Time elapsed: 0.26
---
Combination size: 4
Time elapsed: 1.20
---
Combination size: 5
Time elapsed: 6.62
---
Combination size: 6
Time elapsed: 35.40


In [8]:
b = [[key, value[0], sum(value[1])*a.delta] for key, value in a.packs_pdfs.items()]
b.sort(key = lambda x: x[1])
print('Number of combinations: %d' % (len(a.packs_pdfs.items())))
print('Number of rules: %d' % (len(a.rules)))
b[-5:-1]

Number of combinations: 1787
Number of rules: 167


Time elapsed: 0.00
---
Combination size: 1
Time elapsed: 0.00
---
Combination size: 2
Time elapsed: 0.06
---
Combination size: 3
Time elapsed: 0.26
---
Combination size: 4
Time elapsed: 1.20
---
Combination size: 5
Time elapsed: 6.62
---
Combination size: 6
Time elapsed: 35.40

Number of combinations: 1787
Number of rules: 167


[[['ball', 'blocks', 'blocks', 'blocks', 'book', 'book'],
  37.906162169922766,
  1.0252617203424534],
 [['blocks', 'blocks', 'book', 'horse', 'horse', 'horse'],
  37.910800949200095,
  1.0132149719973149],
 [['ball', 'ball', 'blocks', 'blocks', 'blocks', 'book'],
  38.018974500642763,
  1.0125520827893959],
 [['ball', 'blocks', 'blocks', 'horse', 'horse', 'horse'],
  38.027308293509002,
  1.0006546717326026]]